In [ ]:
## https://github.com/biomag-lab/hypocotyl-UNet

In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
from skimage import io
from itertools import product
from collections import defaultdict
from shutil import copyfile

In [7]:
def make_patches(dataset_path, export_path, patch_size=(52, 52), no_overlap=False):

    dataset_images_path = os.path.join(dataset_path, 'images')
    dataset_masks_path = os.path.join(dataset_path, 'masks')
    new_images_path = os.path.join(export_path, 'images')
    new_masks_path = os.path.join(export_path, 'masks')

    for image_filename in os.listdir(dataset_images_path):

        im = io.imread(os.path.join(dataset_images_path, image_filename))
        masked_im = io.imread(os.path.join(dataset_masks_path, image_filename))

        x_start = list()
        y_start = list()

        if no_overlap:
            x_step = patch_size[0]
            y_step = patch_size[1]
        else:
            x_step = patch_size[0] // 2
            y_step = patch_size[1] // 2

        for x_idx in range(0, im.shape[0] - patch_size[0] + 1, x_step):
            x_start.append(x_idx)

        if im.shape[0] - patch_size[0] - 1 > 0:
            x_start.append(im.shape[0] - patch_size[0] - 1)

        for y_idx in range(0, im.shape[1] - patch_size[1] + 1, y_step):
            y_start.append(y_idx)

        if im.shape[1] - patch_size[1] - 1 > 0:
            y_start.append(im.shape[1] - patch_size[1] - 1)

        for num, (x_idx, y_idx) in enumerate(product(x_start, y_start)):
            new_image_filename = os.path.splitext(image_filename)[0] + '_%d.png' % num

            io.imsave(
                os.path.join(new_images_path, new_image_filename),
                im[x_idx:x_idx + patch_size[0], y_idx:y_idx + patch_size[1], :]
            )

            io.imsave(
                os.path.join(new_masks_path, new_image_filename),
                masked_im[x_idx:x_idx + patch_size[0], y_idx:y_idx + patch_size[1]]
            )

In [8]:
dataset_path = 'data/'
export_path = 'patches/'

In [9]:
make_patches(dataset_path, export_path, patch_size=(52, 52), no_overlap=False)